In [1]:
!pip install google-play-scraper pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from google_play_scraper import Sort, reviews_all # Pastikan Anda sudah menginstal: pip install google-play-scraper
import time
import datetime

In [3]:
print("Memulai proses scraping ulasan Duolingo dari Google Play Store...")

# ID Aplikasi Duolingo
app_id = 'com.duolingo'

# Jumlah ulasan minimal yang ingin dikumpulkan
min_reviews_needed = 10000

# Bahasa ulasan yang ingin difokuskan (misal: Bahasa Indonesia)
# Jika 10,000 tidak tercapai dalam 1 bahasa, Anda bisa mencoba bahasa lain atau menghapus filter bahasa.
lang_filter = 'id' # 'id' untuk Bahasa Indonesia. Hapus baris ini untuk semua bahasa.

# List untuk menyimpan hasil scraping
all_reviews = []

# Menggunakan reviews_all untuk mendapatkan semua ulasan atau sampai batas yang ditentukan
# Proses ini bisa memakan waktu tergantung jumlah ulasan dan koneksi internet Anda.
# Perhatikan rate limit jika Anda mencoba mengumpulkan terlalu banyak terlalu cepat.
try:
    # `reviews_all` mencoba mengumpulkan semua ulasan.
    # Jika volume sangat besar, ini akan berjalan terus.
    # Kita bisa tambahkan limit manual jika ingin mengontrol.

    # Untuk Duolingo, ada jutaan ulasan, jadi kita perlu batas agar tidak berjalan selamanya.
    # Kita akan ambil 10,000 ulasan (sesuai target minimal) dari yang paling relevan (Sort.NEWEST atau Sort.RATING)
    # Sort.NEWEST bisa memberikan ulasan terbaru. Sort.RATING bisa memberikan campuran.
    # Untuk tujuan sentimen, Sort.NEWEST atau Sort.RRATING (tingkat relevansi) bisa bagus.

    print(f"Mengumpulkan ulasan untuk App ID: {app_id}")
    print(f"Target minimal ulasan: {min_reviews_needed}")
    if lang_filter:
        print(f"Memfilter berdasarkan bahasa: {lang_filter}")

    # Mengambil ulasan. Parameter `lang` bisa diatur.
    # Menggunakan generator untuk efisiensi memori
    count = 0
    for r in reviews_all(
        app_id,
        lang=lang_filter if lang_filter else 'en', # Menggunakan 'en' jika lang_filter kosong atau jika tidak ada filter.
        country='id', # Optional: Fokus negara, bisa membantu filter bahasa jika lang tidak cukup
        sort=Sort.NEWEST # Urutkan dari yang terbaru
    ):
        all_reviews.append(r)
        count += 1
        if count % 1000 == 0: # Print progress setiap 1000 ulasan
            print(f"Terkumpul {count} ulasan...")

        if count >= min_reviews_needed: # Hentikan jika sudah mencapai target minimal
            break

        time.sleep(0.1) # Jeda singkat untuk menghindari rate limit

except Exception as e:
    print(f"Terjadi kesalahan saat scraping: {e}")
    print(f"Total ulasan yang berhasil terkumpul sebelum error: {len(all_reviews)}")

print(f"\nProses scraping selesai. Total ulasan terkumpul: {len(all_reviews)}")

# Konversi ke DataFrame
df_reviews = pd.DataFrame(all_reviews)

# Cek apakah ada data
if not df_reviews.empty:
    # Memilih kolom yang relevan: 'content' (teks ulasan) dan 'score' (rating)
    df_reviews_selected = df_reviews[['content', 'score']]

    # Simpan ke file CSV
    # Pastikan file CSV ini akan digunakan sebagai input di notebook analisis sentimen utama
    output_filename = 'duolingo_reviews.csv'
    df_reviews_selected.to_csv(output_filename, index=False)
    print(f"Data ulasan berhasil disimpan ke '{output_filename}'")
    print("\nBerikut 5 baris pertama data yang tersimpan:")
    print(df_reviews_selected.head())
    print(f"\nJumlah total baris data yang tersimpan: {len(df_reviews_selected)}")
else:
    print("Tidak ada ulasan yang terkumpul.")

Memulai proses scraping ulasan Duolingo dari Google Play Store...
Mengumpulkan ulasan untuk App ID: com.duolingo
Target minimal ulasan: 10000
Memfilter berdasarkan bahasa: id
Terkumpul 1000 ulasan...
Terkumpul 2000 ulasan...
Terkumpul 3000 ulasan...
Terkumpul 4000 ulasan...
Terkumpul 5000 ulasan...
Terkumpul 6000 ulasan...
Terkumpul 7000 ulasan...
Terkumpul 8000 ulasan...
Terkumpul 9000 ulasan...
Terkumpul 10000 ulasan...

Proses scraping selesai. Total ulasan terkumpul: 10000
Data ulasan berhasil disimpan ke 'duolingo_reviews.csv'

Berikut 5 baris pertama data yang tersimpan:
                                             content  score
0  bagus dan sangat membantu sangat di rekomendas...      5
1  seru sekali jadi bikin saya bisa belajar bahas...      5
2    apk ini bagus pelajaran yang di dapatkan detail      5
3           saya beri 4 Strar duluh soal Belum lekap      4
4  aplikasi nya cocok untuk orang yang mau belaja...      5

Jumlah total baris data yang tersimpan: 10000


In [4]:
# Muat dataset hasil scraping
df = pd.read_csv('duolingo_reviews.csv')

print("Tampilan 5 baris pertama dataset:")
print(df.head())
print("\n" + "="*50 + "\n")

Tampilan 5 baris pertama dataset:
                                             content  score
0  bagus dan sangat membantu sangat di rekomendas...      5
1  seru sekali jadi bikin saya bisa belajar bahas...      5
2    apk ini bagus pelajaran yang di dapatkan detail      5
3           saya beri 4 Strar duluh soal Belum lekap      4
4  aplikasi nya cocok untuk orang yang mau belaja...      5


